In [2]:
from datasets import load_dataset

imdb = load_dataset("imdb")

c:\Users\ranax\anaconda3\envs\bert_tf\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset imdb (C:/Users/ranax/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)
100%|██████████| 3/3 [00:00<00:00, 94.00it/s]


In [3]:
small_train_dataset = imdb["train"].shuffle(seed=42).select([i for i in list(range(3000))])
small_test_dataset = imdb["test"].shuffle(seed=42).select([i for i in list(range(3000))])
print(small_train_dataset)

Loading cached shuffled indices for dataset at C:/Users/ranax/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1\cache-8a9e43a6ac4acdff.arrow
Loading cached shuffled indices for dataset at C:/Users/ranax/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1\cache-2eff9f118d84c6fe.arrow


Dataset({
    features: ['text', 'label'],
    num_rows: 3000
})


In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [5]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [6]:
# tokenized_imdb = imdb.map(preprocess_function, batched=True)
tokenized_imdb_train = small_train_dataset.map(preprocess_function, batched=True)
tokenized_imdb_test = small_test_dataset.map(preprocess_function, batched=True)

100%|██████████| 3/3 [00:01<00:00,  2.53ba/s]


In [7]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [8]:
from transformers import TFAutoModelForSequenceClassification
import tensorflow as tf

model = TFAutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2
)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['activation_13', 'vocab_projector', 'vocab_transform', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'dropout_19', 'pre_classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [9]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_imdb_train,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_imdb_test,
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [10]:
from transformers import create_optimizer
import tensorflow as tf

batch_size = 16
num_epochs = 5
batches_per_epoch = len(tokenized_imdb_train) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [11]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['activation_13', 'vocab_projector', 'vocab_transform', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'pre_classifier', 'dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use i

In [12]:
import tensorflow as tf

model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [13]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=3)

Epoch 1/3
187/187 [==============================] - 3760s 20s/step - loss: 0.4005 - val_loss: 0.3522
Epoch 2/3
187/187 [==============================] - 3735s 20s/step - loss: 0.1943 - val_loss: 0.2371
Epoch 3/3
187/187 [==============================] - 3954s 21s/step - loss: 0.1044 - val_loss: 0.2667


In [1]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

# You can, of course, use your own username and model name here once you've pushed your model using the code above!
model = TFAutoModelForSequenceClassification.from_pretrained("fassahat/distill-bert-finetuned-150k-patent-sentences")
tokenizer = AutoTokenizer.from_pretrained("fassahat/distill-bert-finetuned-150k-patent-sentences")

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some layers from the model checkpoint at fassahat/distill-bert-finetuned-150k-patent-sentences were not used when initializing TFDistilBertForSequenceClassification: ['dropout_39']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at fassahat/distill-bert-finetuned-150k-patent-sentences and are newly initialized: ['dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/360 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/711k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [8]:
sentences = [
    "According to the trap of the invention, the mouse, on contacting the adhesive pad, will advantageously be held by the adhesive in an optimum position to achieve most effective capture and/or kill (ie in an “optimum capture/kill position”).",
    "In an alternative embodiment, the enclosure comprises two movable parts each in the form of a comb, wherein each movable part comprises a series of spaced apart fingers which are able to interleave when the trap is in the closed state so as to substantially conceal and isolate from the external environment a captured mouse."
]

In [4]:
import numpy as np
tokenized = tokenizer(sentences, return_tensors="np", padding="longest")

outputs = model(tokenized).logits

classifications = np.argmax(outputs, axis=1)
print(classifications)

[2 2]


In [1]:
patent_example_sentences = [
    "The invention provides a trap for capturing and/or killing a mouse, the trap comprising:The present invention relates to mousetraps for trapping and/or killing mice, in particular, domestic mice.",
    "Mice, although small, can still cause much damage.",
    " They destroy articles such as food, books, furniture and even appliances with their gnawing, urine and faeces.",
    " Another undesirable trait associated with mice, particularly in domestic environments, is their association with a variety of human diseases, such as for example, asthma.",
    " Mice can also spread a variety of organisms that can cause diseases of humans and pets.",
    " These include a variety of food poisoning bacteria like Salmonella, Shigella, Escherichia coli (E.",
    " coli), and others.",
    " While the risk of mouse invasion is able to be somewhat controlled or minimised by preventing food waste such as crumbs accumulating in the home, or ensuring that stored food is isolated from the external environment in, for example, sealed containers, it is not always possible to ensure that no food will be left exposed for a period of time.",
    " Also, minimising the risk of mice invasion by sealing of cracks, spaces and openings such as vents, pipes and chimney access points can be an expensive, labour intensive and a time consuming exercise and is not always possible.",
    "Traditional methods of dealing with mouse invasion in the home involves the use of poisoned baits (rodenticides) that contain anticoagulants, such as warfarin, pival, brodifacoum, difethialone and chlorophacinone.",
    " Such baits, while effective, are nevertheless toxic and are therefore undesirable for use in the home, particularly those in which children and pets reside.",
    " In addition, poisoned mice often die in inaccessible locations thereby resulting in an unpleasant odor upon death of the mouse.",
    " To circumvent these problems, mousetraps are often employed instead of poisoned baits.",
    " Traps are generally preferred as these are less hazardous to use around children and pets.",
    " Because mice are caught by the trap, there is less chance of odor from mice dying in wall voids or other inaccessible areas.",
    " Mousetraps in the form of wooden-based snap traps are common.",
    "Whilst most mousetraps are generally effective, they have the problem of exposing the homeowner, for example, to the highly undesirable task of disposing of the dead mouse once trapped.",
    " Another associated disadvantage with conventional mousetrap is that once the mouse has entered the trap and accessed the bait, depending on the mouse response time, the mouse may still be able to escape by responding quickly enough to thereby avoid capture once the trap has been triggered.",
    " Another disadvantage associated with many mousetraps is they allow the possibility of the mouse escaping even after entering the mousetrap as a result of poor trap design or ineffective trap triggering systems.",
    "While recognising the shortcomings of prior art mousetraps, the present inventors have sought to overcome these deficiencies with a mousetrap that is able to readily capture and secure a mouse or the like with little or no possibility of escape.",
    " Moreover, when captured, the mouse is in the best location on the trap to ensure an efficient kill.",
    " Such traps are desirably simple and cost effective to manufacture and are also able to efficiently trap mice.",
    "The invention provides a trap for capturing and/or killing a mouse, the trap comprising:an enclosure comprising a base connected to at least one movable part and which houses a trigger; the trigger comprising an adhesive section;wherein the at least one movable part is urged against the force of a biasing means from a first position wherein the trap is in a closed state into a second position in which the trap is in a set state;wherein the at least one movable part is maintained in the second position by releasably engaging the trigger; andwherein the trap is triggered upon the mouse contacting the adhesive section and elevating the trigger so as to disengage the trigger from the at least one movable part so that it moves from the second position to the first position under the force of the biasing means.",
    "According to the trap of the invention, the mouse, on contacting the adhesive pad, will advantageously be held by the adhesive in an optimum position to achieve most effective capture and/or kill (ie in an “optimum capture/kill position”).",
    " Moreover, by virtue of the adhesive section, for example, an adhesive pad, located on the trigger, the mousetrap is able to be triggered by elevation of the pad by the mouse as opposed to conventional traps in which triggering is often effected by depression of a triggering mechanism.",
    " The novel triggering system of the invention provides a more stable and efficient capture/kill device as the mouse has to be in the “optimum capture/kill position” in order to trigger the mousetrap and so false triggering (ie when non capture or no kill occurs after trigger) is avoided or minimised.",
    "It will be appreciated that the term “closed state” as used in the description encompasses the state of the trap when it is not set (ie is empty or contains a captured mouse).",
    "It will also be appreciated that any type of adhesive that is sufficiently strong to hold the mouse in the “optimum kill position” and ensures lifting of the trigger as the mouse tries to escape is suitable for use in the trap of the invention.",
    " Suitable adhesives include, but are not limited to, high tack hot melt adhesives (block polymers, polyamides) and double sided adhesive tapes or labels.",
    "Whilst the trap according to the invention may be re-set after capture and re-used, desirably, the trap according to the invention is a single-use trap to be disposed after capture and/or killing of a mouse.",
    " Accordingly, it will be appreciated that the trap be easy and inexpensive to manufacture.",
    " Materials suitable for manufacturing the trap include, but are not limited to, polypropylene, ABS, polystyrene and polyethylene.",
    "In one embodiment, the enclosure comprises one movable part in the form of a plate.",
    " Preferably, the plate is hinge-connected to the plate.",
    " Preferably also, the plate is configured so as to substantially conceal and isolate from the external environment a captured mouse.",
    "In an alternative embodiment, the enclosure comprises two movable parts each in the form of a comb, wherein each movable part comprises a series of spaced apart fingers which are able to interleave when the trap is in the closed state so as to substantially conceal and isolate from the external environment a captured mouse.",
    "Particularly in traps which are desired to act as “kill traps” as opposed to “live traps” in which the mouse is simply captured, the one or more or more movable parts of the trap may further comprise one or more projections on an internal surface if the movable part, wherein the one or more projections assist in killing a mouse when the one more movable parts move from the second position to the first position under the force of the biasing means.",
    "Preferably, the one or more projections are in the form of a bar.",
    " The bar may also comprise a series of teeth projecting from its periphery which assist in killing a mouse when the one more movable parts move from the second position to the first position under the force of the biasing means.",
    "The biasing means may be any means capable of returning the one or more movable parts from the second position to the first position.",
    " Preferably, the biasing means is a spring.",
    "Preferably, the at least one movable part is hinge-connected to the base, however, it will be appreciated that other forms of connection are possible, provided the one or more movable parts are able to move from the first position to the second position in relation to the base.",
    "Preferably, the trap according to the invention is a single-use disposable trap which allows a user to easily and hygienically dispose of a trapped mouse.",
    " Additionally, the trap is desirably configured so that the enclosure is able to conceal a trapped mouse to a large extent (ie 70-80%), but still allow a user to see that a mouse is trapped when the trap is in the closed state.",
    "The invention further extends to a method of capturing and/or killing a mouse comprising the use of a trap as hereinbefore described.",
    " The invention also extends the use of a trap according to any one of the preceding claims to capture and/or kill a mouse.",
    "The invention will now be described with reference to alternative and non-limiting embodiments of the invention as depicted in the following figures.",
    "FIG.",
    " 1 shows the trap in perspective view according to one embodiment of the invention.",
    "FIG.",
    " 2 shows the trap of FIG.",
    " 1 in perspective view with part of the enclosure removed so as to more clearly show the trap internal parts.",
    "FIG.",
    " 3 shows the trap of FIG.",
    " 1 in cut-section view showing the trap during the stages between a set position and a closed position.",
    " In particular, FIG.",
    " 3( a) shows the trap in an open (or set) position; FIGS.",
    " 3( b) and 3(c) show the trap in an activated state once triggered (mouse not shown); and FIG.",
    " 3( d) shows the trap in a closed position when the trap is empty.",
    "FIG.",
    " 4 shows the trap in perspective view according to an alternative embodiment of the invention.",
    "FIG.",
    " 5 shows the trap of FIG.",
    " 4 in perspective view with part of the enclosure removed so as to more clearly show the trap internal parts.",
    "FIG.",
    " 6 shows the trap of FIG.",
    " 4 in cut-section view showing the trap during the stages between a set position and a closed position.",
    " In particular, FIG.",
    " 6( a) shows the trap in an open (or set) position; FIG.",
    " 6( b) shows the trap in a closed trap position (mouse not shown); and FIG.",
    " 6( c) shows the trap in a closed position when the trap is empty.",
    "FIG.",
    " 7 shows the trigger of the embodiment shown in FIGS.",
    " 4 to 6.",
    "Referring to FIG.",
    " 1, the trap 100 according to an embodiment of the invention is shown in a set state.",
    " Trap 100 comprises an enclosure formed from a base 105 that is hinge-connected to a movable part wherein the movable part comprises plate 114 and prop 108.",
    " Trap 100 is able to be manually set from a first (closed) position to a second (open) position by moving lever 135 along slot 136 located on the base 105.",
    " Lever 135 is connected to plate 114 and movement of lever 135 along slot 136 acts to move plate 114 from a closed position to an open position to thereby set the trap.",
    "In moving lever 135 along slot 136 to set the trap, plate 114 is urged against the force of a biasing means, spring 120.",
    " As can be more clearly seen in FIG.",
    " 2, spring 120, is connected to plate 114.",
    " In order to trigger trap 100, whereupon plate 114 moves under the force of spring 120 from an open position to a closed position, a mouse in contact with adhesive pad 109 located on trigger 107 (shown more clearly in FIGS.",
    " 2 and 3), lifts the trigger in trying to escape upon contact with the adhesive.",
    "The mechanism of the trigger is more clearly explained with reference to FIGS.",
    " 2 and 3.",
    " Referring to FIG.",
    " 2, this shows the embodiment of FIG.",
    " 1 with part of the enclosure removed for clarity.",
    " Trigger 107, comprising adhesive pad 109, abuts and engages prop 108 (see FIG.",
    " 3) when trap 100 is in a set state ready to trap a mouse.",
    " The mouse, on entry into the set trap 100, will come into contact with the adhesive located on adhesive pad 109 and will be held in this position by the adhesive ready for capture and/or kill.",
    " Capture occurs once the mouse attempts to escape thereby lifting adhesive pad 109 and hence trigger 107 which causes prop 108 to disengage from trigger 107.",
    " Once prop 108 disengages trigger 107, plate 114 is forced from the second (open) position to the first (closed) position to thereby capture the mouse within the enclosure.",
    " In order to assist in killing the mouse, in the embodiment of FIGS.",
    " 1 to 3, plate 114 comprises a series projections each in the form of a bar 115 further comprising a series of teeth 116 which impact or impale a captured mouse as plate 114 moves from the second to the first position.",
    "An alternative embodiment of the trap according to the invention is shown and described with reference to FIGS.",
    " 4 to 7.",
    "Referring to FIG.",
    " 4, trap 200 according to this embodiment of the invention is shown in a set state.",
    " Trap 200 comprises an enclosure formed from a base 205 (shown clearly in FIG.",
    " 5) and two movable parts, wherein base 205 is hinge-connected to the two the movable parts.",
    " Furthermore, each movable part comprises a comb 214 with prop 208 (see FIG.",
    " 6).",
    " Trap 200 is able to be manually set from a first (closed) position to a second (open) position by prising apart combs 214 against the force of a biasing means (spring 220) to thereby engage each prop 208 with clips 210 located on trigger 207 (see FIG.",
    " 7).",
    "In order to trigger trap 200, whereupon combs 214 move under the force of spring 220 from an open position to a closed position, a mouse in contact with adhesive pad 209 located on trigger 207 (shown more clearly in FIGS.",
    " 5 and 7), lifts trigger 207 in trying to escape upon contact with the adhesive.",
    " Upon lifting trigger 207, props 208 are caused to disengage clips 210 located on trigger 207, thereby allowing combs 214 to move from the second (set) position to the first (closed) position under the force of spring 220 thereby capturing the mouse.",
    "In order to assist in killing the mouse, in the embodiment of FIGS.",
    " 4 to 7, combs 214 comprise projections in the form of bars 215 which impact or impale a captured mouse as combs 214 move from the second to the first position.",
    " Whilst not shown in FIGS.",
    " 4 to 7, each bar 215 may further comprise teeth, spikes, or such like which assist in achieving efficient kill of the mouse upon capture.",
    "It will be appreciated by persons skilled in the art that numerous variations and/or modifications may be made to the invention as shown in the specific embodiments described herein without departing from the scope of the invention as broadly described.",
    " The present embodiments are, therefore, to be considered in all respects as illustrative and not restrictive.",
    "1.",
    " A trap for capturing and/or killing a mouse, the trap comprising:\nan enclosure comprising a base connected to movable parts and a housing a trigger;\nthe moveable parts comprising a plate and a prop,\nwherein the prop comprises two ends, a first end of said two ends is pivotally attached to a part of the plate, and the second end of said two ends is compressed against and abuts a part of the trigger when the trap is in a set state,\nwherein the plate is pivotally attached to a part of the base and is movable with respect to the base when the trap is reconfigured from the set state to a closed state in response to the trap being triggered by a mouse,\nwherein the plate has a length defined between a first end of the plate and a second end of the plate,\nthe trigger comprising an adhesive section pivotally connected to a part of the trap, wherein in a first position the adhesive section is substantially parallel to the base and in a second position the adhesive section is positioned more distantly spaced from the base than in the first position,\nwherein the trap further comprises a biasing means which urges the plate towards the base and towards the adhesive section when the trap is reconfigured from the set state to the closed state; and,\nwherein the trap comprises two open ends at opposite ends of the base, one of the two open ends at each of the opposite ends thereof, said two open ends open to an interior of the trap when the trap is in the set state;\nwherein the adhesive section of the trigger is positioned between the two open ends and within the interior of the trap;\nwherein the trap is configured to be triggered upon the mouse contacting the adhesive section and elevating the adhesive section of the trigger away from the base so as to disengage the trigger from the prop so that the prop is no longer compressed against the trigger, and whereinafter and under a force of the biasing means, the plate moves towards the base and towards the adhesive section and trapping and/or killing a mouse between the base and the plate under the force of the biasing means, and when in the closed state, at least 70% of the captured and/or killed mouse is concealed by the enclosure.",
    "2.",
    " A trap according to claim 1 wherein the plate further comprises one or more projections on an internal surface thereof, wherein the one or more projections are configured to assist in killing the mouse when the plate moves, within the enclosure, under the force of the biasing means when the trap is reconfigured from the set position to the closed position.",
    "3.",
    " A trap according to claim 2 wherein the one or more projections are in the form of a bar.",
    "4.",
    " A trap according to claim 3 wherein the bar comprises a series of teeth projecting from a periphery of the bar that are configured to assist in killing the mouse when the plate moves, within the enclosure, under the force of the biasing means.",
    "5.",
    " A trap according to claim 2, wherein the one or more projections further comprise a plurality of spaced apart projections which are present.",
    "6.",
    " A trap according to claim 2, wherein the one or more projections further comprise at least 4 projections which are present.",
    "7.",
    " A trap according to claim 1 wherein the biasing means is a spring.",
    "8.",
    " A trap according to claim 1 wherein the plate is pivotally attached to a part of the base by being hinged to a side of the base.",
    "9.",
    " A trap according to claim 1, wherein the adhesive section of the trigger is located away from each of the open ends of the trap.",
    "10.",
    " A trap according to claim 9, wherein the adhesive section of the trigger is located within the interior of the trap and is approximately equidistant from each of the open ends of the trap.",
    "11.",
    " A trap according to claim 9, wherein a length of the plate is approximately equal to a distance between the two open ends of the trap.",
    "12.",
    " A trap according to claim 1, wherein the adhesive section of the trigger when in the first position is located in a cavity or recess present in the base.",
    "13.",
    " A trap according to claim 12, wherein a top of the adhesive section of the trigger is approximately coplanar with a top surface of the base within the interior of the trap.",
    "14.",
    " A trap according to claim 1, wherein the enclosure, the base, and the plate define a tunnel-like passage between the two open ends of the trap.",
    "15.",
    " A method of capturing and/or killing a mouse comprising trapping and/or killing a mouse in a trap according to claim 1."
]

In [2]:
from transformers import pipeline

classifier = pipeline("text-classification", "fassahat/distill-bert-finetuned-150k-patent-sentences", framework="tf")

Some layers from the model checkpoint at fassahat/distill-bert-finetuned-150k-patent-sentences were not used when initializing TFDistilBertForSequenceClassification: ['dropout_39']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at fassahat/distill-bert-finetuned-150k-patent-sentences and are newly initialized: ['dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
result = classifier(patent_example_sentences)

In [18]:
len(patent_example_sentences)
final_result = {'advantages':[], 'solutions':[], 'problems':[]}
for i in range(len(patent_example_sentences)):
    if (result[i]['score'] >= 0.8 and len(patent_example_sentences[i]) > 10):
        if (result[i]['label'] == '0'):
            final_result['solutions'].append(patent_example_sentences[i])
        elif (result[i]['label'] == '1'):
            final_result['advantages'].append(patent_example_sentences[i])
        else:
            final_result['problems'].append(patent_example_sentences[i])

final_result


{'advantages': [' Because mice are caught by the trap, there is less chance of odor from mice dying in wall voids or other inaccessible areas.',
  'According to the trap of the invention, the mouse, on contacting the adhesive pad, will advantageously be held by the adhesive in an optimum position to achieve most effective capture and/or kill (ie in an “optimum capture/kill position”).',
  ' The novel triggering system of the invention provides a more stable and efficient capture/kill device as the mouse has to be in the “optimum capture/kill position” in order to trigger the mousetrap and so false triggering (ie when non capture or no kill occurs after trigger) is avoided or minimised.',
  'It will also be appreciated that any type of adhesive that is sufficiently strong to hold the mouse in the “optimum kill position” and ensures lifting of the trigger as the mouse tries to escape is suitable for use in the trap of the invention.',
  'Whilst the trap according to the invention may be 